In [ ]:

# 0. 패키지 불러오기 

# - 공통
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
from sklearn.impute import SimpleImputer # 결측치 처리
from sklearn.preprocessing import LabelEncoder # 데이터 인코딩 (범주형 변수)
from sklearn.preprocessing import StandardScaler # 데이터 스케일링 (수치형 변수)
from sklearn.model_selection import train_test_split # 데이터 분할

# - 분류모델인 경우
from sklearn.ensemble import RandomForestClassifier # 분류모델1 (랜덤포레스트)
from xgboost import XGBClassifier # 분류모델2 (XGBoost)
from sklearn.metrics import roc_auc_score # 평가지표 (AUROC)

# # - 회귀모델인 경우
# from sklearn.ensemble import RandomForestRegressor # 회귀모델1 (랜덤포레스트)
# from xgboost import XGBRegressor # 회귀모델2 (XGBoost)
# from sklearn.metrics import mean_squared_error # 평가지표 (MSE)


# ------------------------------------------------------------------------------------------------------------------------------------

# 1. 데이터 불러오기 
# - 문제에 나와 있는 경로 그대로 사용하면 됨
X_test = pd.read_csv("~")
X_train = pd.read_csv("~")
y_train = pd.read_csv("~")

# ------------------------------------------------------------------------------------------------------------------------------------

# 2. 데이터 전처리 
# - 변수 나누기 
# 데이터셋 변수를 자료 타입에 따라 아래 num_cols, cat_cols에 나눠 넣기, 단 ID와 같이 무의미한 값은 넣지 X
num_cols = [] # 변수 중 숫자로 된 변수 넣기
cat_cols = [] # 변수 중 문자로 된 변수 넣기 
X_cols = num_cols + cat_cols

# - 결측치 처리 (결측치 존재하면 최빈값으로 대체, 없으면 아무 처리x)
imp = SimpleImputer(strategy='most_frequent')
X_train[X_cols] = imp.fit_transform(X_train[X_cols])
X_test[X_cols] = imp.fit_transform(X_test[X_cols])

# - 데이터 인코딩 (cat_cols 변수가 하나 이상 존재할 경우만!)
X = pd.concat([X_train, X_test])

for _col in cat_cols:
  le = LabelEncoder()
  le.fit(X[_col])
  X_train[_col] = le.transform(X_train[_col])
  X_test[_col] = le.transform(X_test[_col])

# ------------------------------------------------------------------------------------------------------------------------------------

# 3. 데이터 모형 구축
# - 데이터 분할
X_tr, X_val, y_tr, y_val = train_test_split(X_train[X_cols], y_train, test_size=0.3)

# - 데이터 스케일링 (num_cols 변수가 하나 이상 존재할 경우만!)
sc = StandardScaler()
sc.fit(X_tr[num_cols])
X_tr[num_cols] = sc.transform(X_tr[num_cols])
X_val[num_cols] = sc.transform(X_val[num_cols])
X_test[num_cols] = sc.transform(X_test[num_cols])

# - 모델 학습 (분류모델인 경우)
m1 = RandomForestClassifier()
m1.fit(X_tr, y_tr)
pred1 = m1.predict_proba(X_val)[:, 1]

m2 = XGBClassifier()
m2.fit(X_tr, y_tr)
pred2 = m2.predict_proba(X_val)[:, 1]

# # - 모델 학습 (회귀모델인 경우)
# m1 = RandomForestRegressor()
# m1.fit(X_tr, y_tr)
# pred1 = m1.predict(X_val)

# m2 = XGBRegressor()
# m2.fit(X_tr, y_tr)
# pred2 = m2.predict(X_val)

# ------------------------------------------------------------------------------------------------------------------------------------

# 4. 데이터 모형 평가
# 모델 평가 (분류모델인 경우)
score1 = roc_auc_score(y_val, pred1)
score2 = roc_auc_score(y_val, pred2)
print(score1, score2)
# -> 둘 중 점수가 더 큰 모델 선택

# # 모델 평가 (회귀모델인 경우)
# score1 = mean_squared_error(y_val, pred1)
# score2 = mean_squared_error(y_val, pred2)
# print(score1, score2)
# -> 둘 중 점수가 더 작은 모델 선택 

# ------------------------------------------------------------------------------------------------------------------------------------

# 5. 결과 생성
# 분류 모델인 경우 
pred = m1.predict_proba(X_test[X_cols])[:,1]
pd.DataFrame({'cust_id(문제확인)': X_test.cust_id, '변수명': pred}).to_csv('수험번호.csv', index=False) # 문제 참고

# # 회귀 모델인 경우 
# pred = m1.predict(X_test[X_cols])
# pd.DataFrame({'cust_id(문제확인)': X_test.cust_id, '변수명': pred}).to_csv('수험번호.csv', index=False) # 문제 참고